# Phase 2: Classifier

In [ ]:
seed = 1030#,1,2,3,4,55,666,7777,88888,999999

In [ ]:
!pip install git+https://github.com/marcoancona/DeepExplain

In [ ]:
!pip install kneed

In [ ]:
!pip install shap==0.31

In [ ]:
#!pip uninstall h5py
!pip install h5py==2.10.0

In [ ]:
#!pip install git+https://github.com/albermax/innvestigate
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/My Drive/CNV')

In [ ]:
!ls

In [ ]:
!pip install tensorflow

In [ ]:
print()

In [ ]:

# Load the TensorBoard notebook extension
"""
%load_ext tensorboard
%tensorflow_version 1
#Start tensorboard
%tensorboard — logdir logs
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter('ignore')
"""


In [ ]:

import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
#from tensorflow import set_random_seed
#set_random_seed(seed)

In [ ]:
#!pip install --upgrade scikit-learn
#!pip install --upgrade imblearn

In [ ]:
#! pip install scikit-learn==0.21.1
!pip install --upgrade scikit-learn
! pip install -U imbalanced-learn

In [ ]:
%matplotlib inline  
import tensorflow as tf
import imp
import numpy as np
import os
import keras
import keras.backend
import keras.models
#import innvestigate
#import innvestigate.utils as iutils
#!pip install scikit-learn==0.20.0
import sklearn
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras import regularizers
from keras.models import Model,Sequential
from keras.layers import Input,Dense,Dropout
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.utils import plot_model
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import keras.models as mod
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import TensorBoard
from keras.utils.vis_utils import plot_model
#from sklearn.externals import joblib
import joblib
drpRate = 0.55
tf.compat.v1.disable_eager_execution()

In [ ]:
#%tensorflow_version 1


In [ ]:
#Initially 0
np.random.seed(seed)
df = pd.read_csv('./dataset/TCGA_BRCA_CNV.gz',compression='gzip',sep='\t',index_col=0)
df = df.reindex(sorted(df.columns), axis=1)
df=df.T
print("CNV Matrix Before:", df.shape)
df2=pd.read_csv('./dataset/BRCA_clinicalMatrix.gz',compression='gzip',sep='\t',index_col=0)
print("Clinical Data Before:", df2.shape)
k=df2.columns.get_loc('PAM50Call_RNAseq')
df2=df2[df2.iloc[:,k].isna()==False]
#df2=df2[df2.iloc[:,k]!='Normal']
#d11=list(set(df.index)-set(df2.index)) #Comment this, if you want to test on unknown sample (generalize model)
commonIndexes=list(set(df2.index).intersection(set(df.index))) 
df=df.loc[commonIndexes]
df2=df2.loc[commonIndexes] 
#df=df.drop(d)                         #Comment this, if you want to test on unknown sample (generalize model)
print("CNV After:", df.shape)
print("Clinical Data After:", df2.shape)
print("CNV and Clinical Data corresponds to same Patients: ", False if False in (df.index==df2.index) else True)
unique_elements, counts_elements = np.unique(df2.iloc[:,k], return_counts=True)
print("Labels", unique_elements, counts_elements)

In [ ]:
X=np.array(df.values)
scaler=StandardScaler()
XScaled=X#scaler.fit_transform(X)
print(XScaled.mean(axis = 0), XScaled.std(axis = 0))

In [ ]:
#Encoding labels to numerical  values
SampleIDs = np.array(df2.index )
Y_temp=np.array(df2.values[:,k])
print("TARGET LABEL SIZE BEFORE: ", Y_temp.shape)
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(Y_temp)
print("TARGET LABEL SIZE BEFORE: ", Y.shape)
print("Labels before and after:", Y_temp[0:9], Y[0:9])
le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(le_name_mapping)

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)
SampleIDs = SampleIDs.reshape(len(Y), 1)
Y = Y.reshape(len(Y), 1)
Y_onehot = onehot_encoder.fit_transform(Y)
print(Y_onehot.shape, SampleIDs.shape)

In [ ]:
#sm=SMOTE(random_state=seed)
from imblearn.over_sampling import ADASYN
sm = ADASYN(random_state=seed)

#X_train1, Y_train1 = X_train, Y_train
#X_train, X_val,Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=seed) ##################3CHANGE####################################
X_train1, Y_train1 = XScaled, Y_onehot
X_train, X_val,Y_train, Y_val = X_train1, [],Y_train1, []     # train_test_split(X_train1, Y_train1, test_size = 0.10, random_state=seed, stratify=Y) ##################3CHANGE####################################
sm=SMOTE(random_state=seed)
X_train_res, Y_train_res= X_train, Y_train   #sm.fit_resample(X_train, Y_train)
X_train_res.shape, Y_train_res.shape

In [ ]:

from keras.models import model_from_json
#json_file=open("./results/encoder.json",'r')  #Remove fitted for generalized learning
json_file=open("SavedModels/encoderBN"+str(seed)+".json",'r')  #Remove fitted for generalized learning
loaded_model_json = json_file.read()
json_file.close()
encoder2 = model_from_json(loaded_model_json)  # for vae only: {'latent_dim': 500})
# load weights into new model

#encoder2.load_weights("./SavedModels/encoder.h5")   #Remove fitted for generalized learning
encoder2.load_weights("SavedModels/encoderBN"+str(seed)+".h5")   #Remove fitted for generalized learning
encoded_data_train=encoder2.predict(X_train_res)


In [ ]:
plt.hist(encoded_data_train[:,9])

In [ ]:
???

In [ ]:
seed

In [ ]:
from keras.layers import Softmax

drpRate1, drpRate2 = 0.4, 0.0 #### cahnge just made in july 2020 only for testing purpose
np.random.seed(seed)


input_dim=encoded_data_train.shape[1]

input_layer2=Input(shape=(input_dim,),name='inputNN')
#hidden11=Dense(100,activation='relu')(input_layer2)
#hidden13=Dense(250,activation='relu')(input_layer2)
hidden132=BatchNormalization()(input_layer2)  #####################################no bn layer
hidden12=Dropout(drpRate1, seed=seed)(hidden132)
hidden14=Dense(200,activation='relu')(hidden12)##############################300
########################################################################hidden132=BatchNormalization()(hidden14)  #remove leadto 72
hidden15=Dropout(drpRate2)(hidden14)
#hidden10=Dense(120,activation='relu')(hidden15)
#hidden101=BatchNormalization()(hidden10)
#hidden11=Dropout(drpRate)(hidden101)
#hidden16=Dense(50,activation='relu')(hidden15)
#########hidden17=Dense(25,activation='relu')(hidden15)#########################25
#hidden162=BatchNormalization()(hidden16)
#hidden161=Dropout(0.5)(hidden162)
output2=Dense(5, name='outputNN')(hidden15)
softmax = Softmax()(output2)
classifier=Model(inputs=input_layer2, outputs=softmax)

print(classifier.summary())



In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(np.argmax(Y_train, axis=-1)),
                                                y=np.argmax(Y_train, axis=-1))



In [ ]:
adam=Adam(lr=0.00002)
classifier.compile(optimizer=adam, loss='categorical_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', min_delta = 0.001, mode='min', verbose=1,patience=250)    #best - 250
tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1)
filepath="./SavedModels/weights-best"+str(seed)+".hdf5"
checkpoint = ModelCheckpoint(filepath, save_weights_only=True, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Fit the model
#model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, callbacks=callbacks_list, verbose=0)

history=classifier.fit(encoded_data_train,Y_train_res,epochs=900,batch_size=64,shuffle=True,       #########################64  ->128(72 accuracy)
                         #When checking whether the model is working or not, set the value to 0.1
                     callbacks=[checkpoint])#, class_weight=class_weights)
#classifier.load_weights(filepath)



In [ ]:
plt.plot(history.history['accuracy'])

plt.show()

In [ ]:
#0

#without stratify-   97-78 0.0,       97, 79- 0.2\\\\//////
#with stratify-      72,99      0.0 xxnormalxxin last:  75, 93 xxnormalxx  0.2 in last

In [ ]:
# for reporting, report accuracy of x_train and not over sampled data
# rerun  testval -> autoencoder2 -> subtype
score=classifier.evaluate(encoded_data_train,Y_train_res)
print("\nTraining Accuracy %s: %.2f%%" % (classifier.metrics_names[1], score[1]*100))



In [ ]:
encoded_data=encoder2.predict(X_train1)
y_pred=classifier.predict(encoded_data)
y_pred=np.argmax(y_pred,axis=-1).reshape(-1,1)
y_true=np.argmax(Y_train1,axis=-1).reshape(-1,1)
#y_pred.shape,y_true.shape
cm=confusion_matrix(y_pred,y_true)
cm.T

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
!pip install mlxtend --upgrade --no-deps 
from mlxtend.plotting import plot_confusion_matrix
import numpy as np
plt.figure()
fig, ax = plot_confusion_matrix(conf_mat=cm.T,
                                colorbar=True,
                                show_absolute=True,
                                show_normed=True, class_names = ['Basal', 'Her2', 'LumA', 'LumB', 'Normal'])
#plt.show()
#plt.savefig('/content/drive/My Drive/BRCA/CMPhase1.png', bbox_inches = 'tight', dpi = 600)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

In [ ]:
classifier.layers

In [ ]:

input_data = Input(shape=(24776,))

encoder_layer1=encoder2.layers[1]
encoder_layer2=encoder2.layers[2]
encoder_layer3=encoder2.layers[3]


#encoder_layer6=encoder2.layers[6]


encoder_layer13 = classifier.layers[1]
encoder_layer14 = classifier.layers[2]
encoder_layer15 = classifier.layers[3]
encoder_layer16 = classifier.layers[4]
encoder_layer17 = classifier.layers[5]
encoder_layer18 = classifier.layers[6]
#encoder_layer19 = classifier.layers[7]



encoder = Model(input_data, encoder_layer17(encoder_layer16(encoder_layer15(encoder_layer14(encoder_layer13(encoder_layer3(encoder_layer2(encoder_layer1(input_data)))))))))
#encoderTemp= Model(input_data, encoder_layer17(encoder_layer16(encoder_layer15(encoder_layer14(encoder_layer13(encoder_layer5(encoder_layer4(encoder_layer3(encoder_layer2(encoder_layer1(input_data)))))))))))

encoder.summary()

In [ ]:
final=encoder
final_json=final.to_json()
with open("./models/CombinedModel_Seed"+str(seed)+".json","w") as json_file:
  json_file.write(final_json)

final.save_weights("./models/CombinedModel_Seed"+str(seed)+".h5")

In [ ]:
tf.reset_default_graph()

In [ ]:
from keras.models import model_from_json

with open("./models/CombinedModel_Seed"+str(seed)+".json", 'r') as json_file:
    encoder= json_file.read()

In [ ]:
encoder = keras.models.model_from_json(encoder)
encoder.summary()

In [ ]:
encoder.load_weights("./models/CombinedModel_Seed"+str(seed)+".h5")

In [ ]:
encoder.layers

In [ ]:
seed

##################################

In [ ]:
tf. __version__

In [ ]:
XScaled, Y_onehot = X_train, Y_train

In [ ]:
from kneed import KneeLocator
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

In [ ]:
encoder.layers[-2]._inbound_nodes

In [ ]:
zero_pos_counter = {
    'deeplift Basal' : 0,
    'intgrad Basal' : 0,
    'grad*input Basal' : 0,
    'elrp Basal' : 0,
    'gradshap Basal': 0,
    'deepshap Basal' : 0,

    'deeplift Her2' : 0,
    'intgrad Her2' : 0,
    'grad*input Her2' : 0,
    'elrp Her2' : 0,
    'gradshap Her2': 0,
    'deepshap Her2' : 0,

    'deeplift LumA' : 0,
    'intgrad LumA' : 0,
    'grad*input LumA' : 0,
    'elrp LumA' : 0,
    'gradshap LumA': 0,
    'deepshap LumA' : 0,

    'deeplift LumB' : 0,
    'intgrad LumB' : 0,
    'grad*input LumB' : 0,
    'elrp LumB' : 0,
    'gradshap LumB': 0,
    'deepshap LumB' : 0,

    'deeplift Normal' : 0,
    'intgrad Normal' : 0,
    'grad*input Normal' : 0,
    'elrp Normal' : 0,
    'gradshap Normal': 0,
    'deepshap Normal' : 0, 
}

In [ ]:
zero_neg_counter = {
    'deeplift Basal' : 0,
    'intgrad Basal' : 0,
    'grad*input Basal' : 0,
    'elrp Basal' : 0,
    'gradshap Basal': 0,
    'deepshap Basal' : 0,

    'deeplift Her2' : 0,
    'intgrad Her2' : 0,
    'grad*input Her2' : 0,
    'elrp Her2' : 0,
    'gradshap Her2': 0,
    'deepshap Her2' : 0,

    'deeplift LumA' : 0,
    'intgrad LumA' : 0,
    'grad*input LumA' : 0,
    'elrp LumA' : 0,
    'gradshap LumA': 0,
    'deepshap LumA' : 0,

    'deeplift LumB' : 0,
    'intgrad LumB' : 0,
    'grad*input LumB' : 0,
    'elrp LumB' : 0,
    'gradshap LumB': 0,
    'deepshap LumB' : 0,

    'deeplift Normal' : 0,
    'intgrad Normal' : 0,
    'grad*input Normal' : 0,
    'elrp Normal' : 0,
    'gradshap Normal': 0,
    'deepshap Normal' : 0, 
}

In [ ]:
sensitivity = 0.9#1e-1

In [ ]:
sensitivity

In [ ]:
encoder.summary()

In [ ]:
XScaled.shape, Y_onehot.shape

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

In [ ]:
from deepexplain.tensorflow import DeepExplain
from collections import Counter
model = encoder
for method in ['deeplift', 'elrp', 'grad*input',  'intgrad']:#, 'saliency']:  
          
          with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
              #model
              input_tensor = model.layers[0].input
              fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
              target_tensor = fModel(input_tensor) 
              for i,subtype in enumerate(['Basal', 'Her2', 'LumA', 'LumB', 'Normal']):
                      IG={}
                      mask = Y_onehot[:,i]==1
                      xs = XScaled[mask]
                      ys = Y_onehot[mask]
                      
                      attributions_ig= de.explain(method, target_tensor, input_tensor, xs, ys=ys)
                      
                      data=np.asarray(attributions_ig)
                      data1 = pd.DataFrame(data, columns=df.columns)
                      
                      genes_union_pos, genes_union_neg = [], []
                      for index, row in data1.iterrows():
                                                 
                        pos = row[row > 0]
                        neg = row[row < 0]*-1
                        
                        indexes=np.argsort(pos)[::-1]
                        genespos=pos.index[indexes]
                        relpos=pos.iloc[indexes]
                        if len(relpos) > 1:
                          kneedle_pos = KneeLocator(range(len(relpos)), relpos,   curve="convex", direction="decreasing") #  
                          if kneedle_pos.knee is not None:
                            pos_knee = kneedle_pos.knee
                          else:
                            pos_knee = 0
                            zero_pos_counter[method+' '+subtype] += 1
                        elif len(relpos) == 1:
                          pos_knee = 1
                        else:
                          zero_pos_counter[method+' '+subtype] += 1
                          pos_knee = 0
                        
                        indexes=np.argsort(neg)[::-1]
                        genesneg=neg.index[indexes]
                        relneg=neg.iloc[indexes]
                        if len(relneg) > 1:
                          kneedle_neg = KneeLocator(range(len(relneg)), relneg,   curve="convex", direction="decreasing")
                          if kneedle_neg.knee is not None:
                            neg_knee = kneedle_neg.knee
                          else:
                            neg_knee = 0
                            zero_neg_counter[method+' '+subtype] += 1
                        elif len(relneg) == 1:
                          neg_knee = 1
                        else:
                          zero_neg_counter[method+' '+subtype] += 1
                          neg_knee = 0
                        
                        #kneedle_pos.plot_knee()
                        #print(kneedle_pos.knee, kneedle_neg.knee)
                        #kneedle_neg.plot_knee()
                                                
                        genes_union_pos.extend(list(relpos.head(pos_knee).index))
                        genes_union_neg.extend(list(relneg.head(neg_knee).index))
                        

                      genes_count_pos = dict(Counter(genes_union_pos))
                      genes_count_neg = dict(Counter(genes_union_neg))

                      freq_pos = dict(sorted(genes_count_pos.items(), key=lambda item: item[1], reverse=True))
                      freq_df_pos = pd.DataFrame(freq_pos.items(), columns=[subtype+'Genes', 'Frequency'])
                      freq_neg = dict(sorted(genes_count_neg.items(), key=lambda item: item[1], reverse=True))
                      freq_df_neg = pd.DataFrame(freq_neg.items(), columns=[subtype+'Genes', 'Frequency'])
                     
                      final_pos = freq_df_pos  #[freq_df_pos['Frequency'] >= round(data1.shape[0]*.2)]
                      print(method, subtype, final_pos.shape, 'pos')
                      final_neg = freq_df_neg  #[freq_df_neg['Frequency'] >= round(dat  `a1.shape[0]*.2)]
                      print(method, subtype, final_neg.shape, 'neg')

                      final_pos.to_csv('./Attributions68NoNORMDRP0 Patientwise/seed'+str(seed)+'/'+method+'_'+str(subtype)+'_pos.csv')
                      final_neg.to_csv('./Attributions68NoNORMDRP0 Patientwise/seed'+str(seed)+'/'+method+'_'+str(subtype)+'_neg.csv')
                      
              #bottom100.T.plot.bar()